In [85]:
import os
import re
import pandas as pd
import numpy as np

from models import *

from numpy.linalg import norm
from ast import literal_eval
from operator import itemgetter

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf

In [86]:
global graph
graph = tf.get_default_graph()

In [87]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [88]:
# to save space in the CSV, only track non-zero values in the features;
# full lists can be reconstructed later
def nonzeroes(arr, filtered):
    for idx, item in enumerate(arr):
        if type(item) is np.ndarray:
            filtered[idx] = nonzeroes(item, dict())
        else:
            if item != 0:
                filtered[idx] = item
    return filtered



In [96]:
property_data = pd.read_csv('static/data/properties2.csv')
property_data = property_data['MLSNUM']

In [111]:
## create a regex pattern
p = re.compile('(\d{8})_img_(\d+).jpg')

# create an array to hold image features which will be turned into a dataframe 
# (faster than starting with a df)
data_arr = []
piccount = 0
failcount = 0

# actual image jpgs are not kept within project directory as they would take up too much space
for root, dirs, files in os.walk('/Users/davidcrowe/Desktop/imgs2'):
    for name in files:
        piccount += 1
        print(piccount)
        match = p.match(name)
        if match:
            try:
                img_dict = {
                    'MLSNUM': match.group(1),
                    'IMGNUM': match.group(2)
                }
                path = os.path.join(root, name)
                img = image.load_img(path, target_size=(224,224))
                img_data = image.img_to_array(img)
                img_data = np.expand_dims(img_data, axis=0)
                img_data = preprocess_input(img_data)
                
                with graph.as_default():
                    features = model.predict(img_data)

                img_dict['FEATURES'] = nonzeroes(features, dict())
               
                data_arr.append(img_dict)
            except OSError:
                failcount += 1
                
print('There were ' + str(failcount) + ' images that failed to load.')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [101]:
data_df = pd.DataFrame(data_arr)
data_df.shape

(0, 0)

In [91]:
data_df.head(100)

,MLSNUM,IMGNUM,FEATURES
0,71835057,14,"{0: {0: {0: {35: 0.66513985, 44: 3.304565, 47:..."
1,71857486,14,"{0: {0: {0: {15: 4.1395807, 26: 7.2350492, 28:..."
2,71826868,1,"{0: {0: {0: {14: 1.6307721, 35: 0.33817983, 44..."
3,71857097,22,"{0: {0: {0: {17: 38.6713, 22: 6.0093694, 25: 0..."
4,71901013,10,"{0: {0: {0: {12: 4.680094, 42: 3.998118, 45: 2..."
5,71787968,17,"{0: {0: {0: {4: 28.705866, 28: 43.883663, 88: ..."
6,71823185,17,"{0: {0: {0: {28: 7.171619, 45: 0.7569923, 51: ..."
7,71794382,5,"{0: {0: {0: {30: 1.0088907, 46: 1.2560357, 54:..."
8,71620096,12,"{0: {0: {0: {25: 47.67246, 28: 0.21037275, 63:..."
9,71797994,21,"{0: {0: {0: {15: 0.0663915, 28: 6.11286, 50: 1..."


In [92]:
data_df['FEATURES'].iloc[0]

{0: {0: {0: {35: 0.66513985,
    44: 3.304565,
    47: 9.166828,
    113: 3.7093382,
    123: 2.8831832,
    125: 2.1549535,
    131: 10.861174,
    152: 15.2498045,
    155: 5.349974,
    168: 17.952564,
    221: 7.2979183,
    224: 0.66750073,
    230: 5.988727,
    236: 5.951447,
    272: 18.767445,
    276: 26.575792,
    315: 0.55368125,
    342: 8.204115,
    355: 8.772916,
    382: 59.06855,
    390: 2.5087187,
    425: 0.27467307,
    435: 6.3200703,
    444: 7.3550715,
    467: 3.153195,
    477: 5.5907183,
    480: 33.227055,
    483: 2.1611907},
   1: {15: 10.811149,
    44: 1.7973429,
    58: 1.8344502,
    125: 27.032787,
    131: 2.074884,
    152: 14.72678,
    162: 8.819972,
    168: 39.270863,
    177: 3.5876486,
    192: 4.581216,
    221: 11.3883,
    227: 9.892885,
    229: 0.90289456,
    230: 25.04329,
    267: 1.3920524,
    272: 58.836357,
    276: 70.820694,
    317: 5.948308,
    324: 2.8165433,
    342: 2.7402854,
    352: 2.6144888,
    354: 8.452511,
    35

In [93]:
## export the used information to a csv file
data_df.to_csv('static/data/feature_data_df.csv', index=None)

In [112]:
mlsNums = data_df["MLSNUM"]
mlsNums.head

KeyError: 'MLSNUM'